In [1]:
import pandas as pd
import numpy as np

In [2]:
X = pd.read_csv('train.csv')
#Y = X.pop('SalePrice')
X.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
X.shape

(1460, 81)

In [4]:
null = X.isnull().sum()
null_values = null[null>0]
null_values_perc = null_values/X.shape[0]

In [5]:
feature_perc = X.nunique()/X.shape[0]
unique_features = feature_perc[feature_perc>0.7]
unique_features

Id         1.000000
LotArea    0.734932
dtype: float64

In [6]:
redundant_features = pd.concat([null_values_perc[null_values_perc>0.5], unique_features])

In [7]:
redundant_features

Alley          0.937671
PoolQC         0.995205
Fence          0.807534
MiscFeature    0.963014
Id             1.000000
LotArea        0.734932
dtype: float64

Will have to drop the following features, according to the percentage of the data that is missing

Alley:          0.937671

PoolQC:        0.995205

Fence:          0.807534

MiscFeature:    0.963014

And these according to the number of unqi=

In [8]:
X = X.drop(redundant_features.index, axis = 1)

In [9]:
X.head()

,MSSubClass,MSZoning,LotFrontage,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,20,RL,80.0,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,60,RL,68.0,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,70,RL,60.0,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,...,0,0,0,0,0,12,2008,WD,Normal,250000


In [10]:
two_features = X.nunique() <= 2

In [11]:
two_features[two_features]

Street        True
Utilities     True
CentralAir    True
dtype: bool

In [12]:
X[['Street', 'Utilities', 'CentralAir']].head()

,Street,Utilities,CentralAir
0,Pave,AllPub,Y
1,Pave,AllPub,Y
2,Pave,AllPub,Y
3,Pave,AllPub,Y
4,Pave,AllPub,Y


In [13]:
two_dummies = pd.get_dummies(X[['Street', 'Utilities', 'CentralAir']])

In [14]:
two_dummies.sum()

Street_Grvl            6
Street_Pave         1454
Utilities_AllPub    1459
Utilities_NoSeWa       1
CentralAir_N          95
CentralAir_Y        1365
dtype: int64

Will drop the Utilities and Street feature because there is only 1/6 instance in which it is not "AllPub"/Grvl, which I would remove due to lack of a representative number of instances. 

Leaving the case where it is always Grvl or AllPub so they can be removed anyway

In [15]:
X = X.drop(['Street', 'Utilities'], axis = 1)

Will now one_hot_encode Central Air

In [16]:
dummy = pd.get_dummies(X['CentralAir'])
X = pd.concat([X.drop('CentralAir', axis=1), dummy], axis =1)

In [17]:
X.head()

,MSSubClass,MSZoning,LotFrontage,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,...,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,N,Y
0,60,RL,65.0,Reg,Lvl,Inside,Gtl,CollgCr,Norm,Norm,...,0,0,0,2,2008,WD,Normal,208500,0,1
1,20,RL,80.0,Reg,Lvl,FR2,Gtl,Veenker,Feedr,Norm,...,0,0,0,5,2007,WD,Normal,181500,0,1
2,60,RL,68.0,IR1,Lvl,Inside,Gtl,CollgCr,Norm,Norm,...,0,0,0,9,2008,WD,Normal,223500,0,1
3,70,RL,60.0,IR1,Lvl,Corner,Gtl,Crawfor,Norm,Norm,...,0,0,0,2,2006,WD,Abnorml,140000,0,1
4,60,RL,84.0,IR1,Lvl,FR2,Gtl,NoRidge,Norm,Norm,...,0,0,0,12,2008,WD,Normal,250000,0,1


In [18]:
X.head()

,MSSubClass,MSZoning,LotFrontage,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,...,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,N,Y
0,60,RL,65.0,Reg,Lvl,Inside,Gtl,CollgCr,Norm,Norm,...,0,0,0,2,2008,WD,Normal,208500,0,1
1,20,RL,80.0,Reg,Lvl,FR2,Gtl,Veenker,Feedr,Norm,...,0,0,0,5,2007,WD,Normal,181500,0,1
2,60,RL,68.0,IR1,Lvl,Inside,Gtl,CollgCr,Norm,Norm,...,0,0,0,9,2008,WD,Normal,223500,0,1
3,70,RL,60.0,IR1,Lvl,Corner,Gtl,Crawfor,Norm,Norm,...,0,0,0,2,2006,WD,Abnorml,140000,0,1
4,60,RL,84.0,IR1,Lvl,FR2,Gtl,NoRidge,Norm,Norm,...,0,0,0,12,2008,WD,Normal,250000,0,1


In [31]:
X.isnull()

,MSSubClass,MSZoning,LotFrontage,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,...,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,N,Y
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [41]:
X_null = X.isnull().sum()

MSSubClass         0
MSZoning           0
LotFrontage      259
LotShape           0
LandContour        0
LotConfig          0
LandSlope          0
Neighborhood       0
Condition1         0
Condition2         0
BldgType           0
HouseStyle         0
OverallQual        0
OverallCond        0
YearBuilt          0
YearRemodAdd       0
RoofStyle          0
RoofMatl           0
Exterior1st        0
Exterior2nd        0
MasVnrType         8
MasVnrArea         8
ExterQual          0
ExterCond          0
Foundation         0
BsmtQual          37
BsmtCond          37
BsmtExposure      38
BsmtFinType1      37
BsmtFinSF1         0
                ... 
HalfBath           0
BedroomAbvGr       0
KitchenAbvGr       0
KitchenQual        0
TotRmsAbvGrd       0
Functional         0
Fireplaces         0
FireplaceQu      690
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageCars         0
GarageArea         0
GarageQual        81
GarageCond        81
PavedDrive         0
WoodDeckSF   

In [42]:
X_null[X_null > 0]

LotFrontage     259
MasVnrType        8
MasVnrArea        8
BsmtQual         37
BsmtCond         37
BsmtExposure     38
BsmtFinType1     37
BsmtFinType2     38
Electrical        1
FireplaceQu     690
GarageType       81
GarageYrBlt      81
GarageFinish     81
GarageQual       81
GarageCond       81
dtype: int64

Next step is to impute all the missing values

In [48]:
X['Electrical'].value_counts()

SBrkr    1334
FuseA      94
FuseF      27
FuseP       3
Mix         1
Name: Electrical, dtype: int64